In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
from datetime import timedelta
from dateutil.parser import parse
import time
from sklearn.cross_validation import train_test_split

C:\Users\AISWARRIYA\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
import sys
!{sys.executable} -m pip install xgboost

You are using pip version 18.0, however version 18.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [3]:
import xgboost as xgb

In [4]:
dec_org=pd.read_csv("walnut_expense_report_01-12-2018_to_31-12-2018.csv", thousands=',')
dec_org.head()

,DATE,TIME,PLACE,AMOUNT,ACCOUNT,EXPENSE,CATEGORY,TAGS,NOTE
0,01/12/2018,1:00 PM,WESTSIDE,1400.0,CitiBank credit 6328,Yes,SHOPPING,NaN,NaN
1,01/12/2018,1:26 PM,VIMAL SILKS,890.0,CitiBank credit 6328,Yes,SHOPPING,NaN,NaN
2,01/12/2018,1:44 PM,PATANJALI AYURVEDIC & ORGANIC STORE,487.0,CitiBank credit 6328,Yes,SHOPPING,NaN,NaN
3,01/12/2018,1:52 PM,PATANJALI AYURVEDIC & ORGANIC STORE,140.0,CitiBank credit 6328,Yes,SHOPPING,NaN,NaN
4,01/12/2018,1:57 PM,PATANJALI AYURVEDIC & ORGANIC STORE,170.0,CitiBank credit 6328,Yes,SHOPPING,NaN,NaN


In [5]:
dec_org.columns

Index(['DATE', 'TIME', 'PLACE', 'AMOUNT', 'ACCOUNT', 'EXPENSE', 'CATEGORY',
       'TAGS', 'NOTE'],
      dtype='object')

In [6]:
dec_org.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53 entries, 0 to 52
Data columns (total 9 columns):
DATE        53 non-null object
TIME        53 non-null object
PLACE       53 non-null object
AMOUNT      53 non-null float64
ACCOUNT     53 non-null object
EXPENSE     53 non-null object
CATEGORY    52 non-null object
TAGS        20 non-null object
NOTE        22 non-null object
dtypes: float64(1), object(8)
memory usage: 3.8+ KB


In [7]:
dec_org.isnull().sum()

DATE         0
TIME         0
PLACE        0
AMOUNT       0
ACCOUNT      0
EXPENSE      0
CATEGORY     1
TAGS        33
NOTE        31
dtype: int64

In [8]:
X_dec=dec_org.drop(columns=['TIME','EXPENSE','TAGS','NOTE'],axis=1)
X_dec.head(2)

,DATE,PLACE,AMOUNT,ACCOUNT,CATEGORY
0,01/12/2018,WESTSIDE,1400.0,CitiBank credit 6328,SHOPPING
1,01/12/2018,VIMAL SILKS,890.0,CitiBank credit 6328,SHOPPING


In [9]:
dec_org['outcome'] = dec_org.EXPENSE.map({'No':0, 'Yes':1})
dec_org['outcome'].head(10)

0    1
1    1
2    1
3    1
4    1
5    1
6    0
7    1
8    0
9    1
Name: outcome, dtype: int64

In [10]:
y_dec=dec_org['outcome']
y_dec.tail()

48    1
49    1
50    1
51    0
52    1
Name: outcome, dtype: int64

In [11]:
X_dec['DATE1'] = pd.to_datetime(X_dec['DATE'], format='%d/%m/%Y',utc=True)
X_dec['DATE1'].head(10)

0   2018-12-01 00:00:00+00:00
1   2018-12-01 00:00:00+00:00
2   2018-12-01 00:00:00+00:00
3   2018-12-01 00:00:00+00:00
4   2018-12-01 00:00:00+00:00
5   2018-12-01 00:00:00+00:00
6   2018-12-03 00:00:00+00:00
7   2018-12-03 00:00:00+00:00
8   2018-12-05 00:00:00+00:00
9   2018-12-05 00:00:00+00:00
Name: DATE1, dtype: datetime64[ns, UTC]

In [12]:
X_dec.head()

,DATE,PLACE,AMOUNT,ACCOUNT,CATEGORY,DATE1
0,01/12/2018,WESTSIDE,1400.0,CitiBank credit 6328,SHOPPING,2018-12-01 00:00:00+00:00
1,01/12/2018,VIMAL SILKS,890.0,CitiBank credit 6328,SHOPPING,2018-12-01 00:00:00+00:00
2,01/12/2018,PATANJALI AYURVEDIC & ORGANIC STORE,487.0,CitiBank credit 6328,SHOPPING,2018-12-01 00:00:00+00:00
3,01/12/2018,PATANJALI AYURVEDIC & ORGANIC STORE,140.0,CitiBank credit 6328,SHOPPING,2018-12-01 00:00:00+00:00
4,01/12/2018,PATANJALI AYURVEDIC & ORGANIC STORE,170.0,CitiBank credit 6328,SHOPPING,2018-12-01 00:00:00+00:00


In [13]:
X_dec['DATE2']=X_dec['DATE1'].apply(lambda x: x.toordinal())
X_dec['DATE2'].head()

0    737029
1    737029
2    737029
3    737029
4    737029
Name: DATE2, dtype: int64

In [14]:
X_dec.head()

,DATE,PLACE,AMOUNT,ACCOUNT,CATEGORY,DATE1,DATE2
0,01/12/2018,WESTSIDE,1400.0,CitiBank credit 6328,SHOPPING,2018-12-01 00:00:00+00:00,737029
1,01/12/2018,VIMAL SILKS,890.0,CitiBank credit 6328,SHOPPING,2018-12-01 00:00:00+00:00,737029
2,01/12/2018,PATANJALI AYURVEDIC & ORGANIC STORE,487.0,CitiBank credit 6328,SHOPPING,2018-12-01 00:00:00+00:00,737029
3,01/12/2018,PATANJALI AYURVEDIC & ORGANIC STORE,140.0,CitiBank credit 6328,SHOPPING,2018-12-01 00:00:00+00:00,737029
4,01/12/2018,PATANJALI AYURVEDIC & ORGANIC STORE,170.0,CitiBank credit 6328,SHOPPING,2018-12-01 00:00:00+00:00,737029


In [15]:
X_dec.drop(columns=['DATE','DATE1'],axis=1,inplace=True)
X_dec.head()

,PLACE,AMOUNT,ACCOUNT,CATEGORY,DATE2
0,WESTSIDE,1400.0,CitiBank credit 6328,SHOPPING,737029
1,VIMAL SILKS,890.0,CitiBank credit 6328,SHOPPING,737029
2,PATANJALI AYURVEDIC & ORGANIC STORE,487.0,CitiBank credit 6328,SHOPPING,737029
3,PATANJALI AYURVEDIC & ORGANIC STORE,140.0,CitiBank credit 6328,SHOPPING,737029
4,PATANJALI AYURVEDIC & ORGANIC STORE,170.0,CitiBank credit 6328,SHOPPING,737029


In [16]:
categorical_columns = [x for x in X_dec.dtypes.index if X_dec.dtypes[x]=='object']
categorical_columns

['PLACE', 'ACCOUNT', 'CATEGORY']

In [17]:
cols = ['PLACE','ACCOUNT','CATEGORY']
for name in cols:
    print(name,':')
    print(X_dec[name].value_counts(),'\n')

PLACE :
UPI FUNDS TRANSFER                             10
PAYTM.COM/PAYTMWALLE                            6
PHONEPEPRIVA68592                               5
ECS                                             4
PATANJALI AYURVEDIC & ORGANIC STORE             3
AMAZON                                          3
JEYACHANDRA                                     2
PAYTM.COM/RECHARGE                              2
VIMAL SILKS                                     1
BSNL                                            1
AQUA GAURD SERVICE WITH FILLTES REPLACEMENT     1
SARAVANA STORE GOLD PALACE                      1
AMAZON PAY                                      1
DEBIT                                           1
FLIPKART                                        1
VIDEOCON D2H                                    1
PAYTM                                           1
SRI VIGNESHWARI AGE                             1
LIC                                             1
PL. A. KRISHNA INN                        

In [18]:
X_dec['CATEGORY']=X_dec.fillna('POCKET MONEY')

In [19]:
X_dec.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53 entries, 0 to 52
Data columns (total 5 columns):
PLACE       53 non-null object
AMOUNT      53 non-null float64
ACCOUNT     53 non-null object
CATEGORY    53 non-null object
DATE2       53 non-null int64
dtypes: float64(1), int64(1), object(3)
memory usage: 2.1+ KB


In [20]:
df=pd.concat([X_dec,pd.get_dummies(X_dec[['PLACE', 'ACCOUNT', 'CATEGORY']],prefix='column name')],axis=1)
df.head()

,PLACE,AMOUNT,ACCOUNT,CATEGORY,DATE2,column name_AMAZON,column name_AMAZON PAY,column name_AQUA GAURD SERVICE WITH FILLTES REPLACEMENT,column name_ATM,column name_BSNL,...,column name_PHONEPEPRIVA68592,column name_PL. A. KRISHNA INN,column name_SARAVANA STORE GOLD PALACE,column name_SRI VIGNESHWARI AGE,column name_THE ROYAL TIBET,column name_UPI FUNDS TRANSFER,column name_VANGA SAPIDALAM MUD MA,column name_VIDEOCON D2H,column name_VIMAL SILKS,column name_WESTSIDE
0,WESTSIDE,1400.0,CitiBank credit 6328,WESTSIDE,737029,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,VIMAL SILKS,890.0,CitiBank credit 6328,VIMAL SILKS,737029,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,PATANJALI AYURVEDIC & ORGANIC STORE,487.0,CitiBank credit 6328,PATANJALI AYURVEDIC & ORGANIC STORE,737029,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,PATANJALI AYURVEDIC & ORGANIC STORE,140.0,CitiBank credit 6328,PATANJALI AYURVEDIC & ORGANIC STORE,737029,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,PATANJALI AYURVEDIC & ORGANIC STORE,170.0,CitiBank credit 6328,PATANJALI AYURVEDIC & ORGANIC STORE,737029,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
df=df.drop(columns=['PLACE','ACCOUNT','CATEGORY'],axis=1)

In [22]:
X_dec=df
X_dec.head()

,AMOUNT,DATE2,column name_AMAZON,column name_AMAZON PAY,column name_AQUA GAURD SERVICE WITH FILLTES REPLACEMENT,column name_ATM,column name_BSNL,column name_DEBIT,column name_ECS,column name_FLIPKART,...,column name_PHONEPEPRIVA68592,column name_PL. A. KRISHNA INN,column name_SARAVANA STORE GOLD PALACE,column name_SRI VIGNESHWARI AGE,column name_THE ROYAL TIBET,column name_UPI FUNDS TRANSFER,column name_VANGA SAPIDALAM MUD MA,column name_VIDEOCON D2H,column name_VIMAL SILKS,column name_WESTSIDE
0,1400.0,737029,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,890.0,737029,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,487.0,737029,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,140.0,737029,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,170.0,737029,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
y_dec.tail()

48    1
49    1
50    1
51    0
52    1
Name: outcome, dtype: int64

In [24]:
X_dec_train, X_dec_test, y_dec_train, y_dec_test = train_test_split(X_dec, y_dec, test_size = .4, random_state=25)

In [25]:
print(X_dec_train.shape)
print(X_dec_test.shape)
print(y_dec_train.shape)
print(y_dec_test.shape)

(31, 64)
(22, 64)
(31,)
(22,)


In [26]:
from sklearn.linear_model import LogisticRegression

In [27]:
logr=LogisticRegression()

In [28]:
logr.fit(X_dec_train, y_dec_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [29]:
X_dec_pred=logr.predict(X_dec_train)
print(X_dec_pred.shape)

(31,)


In [30]:
y_dec_pred= logr.predict(X_dec_test)
print(y_dec_pred.shape)

(22,)


In [31]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_dec_train, X_dec_pred)
confusion_matrix

array([[ 4,  5],
       [ 1, 21]], dtype=int64)

In [32]:
from sklearn.metrics import classification_report
print(classification_report(y_dec_train, X_dec_pred))

             precision    recall  f1-score   support

          0       0.80      0.44      0.57         9
          1       0.81      0.95      0.88        22

avg / total       0.81      0.81      0.79        31



In [33]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_dec_test, y_dec_pred)
confusion_matrix

array([[ 2,  3],
       [ 1, 16]], dtype=int64)

In [34]:
from sklearn.metrics import classification_report
print(classification_report(y_dec_test, y_dec_pred))

             precision    recall  f1-score   support

          0       0.67      0.40      0.50         5
          1       0.84      0.94      0.89        17

avg / total       0.80      0.82      0.80        22



In [35]:
from sklearn import metrics 
print("Logistic Regression model accuracy(in %):", metrics.accuracy_score(y_dec_test, y_dec_pred)*100)

Logistic Regression model accuracy(in %): 81.81818181818183


In [36]:
from sklearn.naive_bayes import GaussianNB 
gnb = GaussianNB() 
gnb.fit(X_dec_train, y_dec_train) 

GaussianNB(priors=None)

In [37]:
y_dec_pred = gnb.predict(X_dec_test)

In [38]:
from sklearn import metrics 
print("Gaussian Naive Bayes model accuracy(in %):", metrics.accuracy_score(y_dec_test, y_dec_pred)*100)

Gaussian Naive Bayes model accuracy(in %): 63.63636363636363


In [39]:
from sklearn.tree import DecisionTreeClassifier
clf_gini = DecisionTreeClassifier(criterion = "gini", random_state = 100,
                               max_depth=3, min_samples_leaf=5)
clf_gini.fit(X_dec_train, y_dec_train)


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=5, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=100,
            splitter='best')

In [40]:
y_dec_pred = clf_gini.predict(X_dec_test)

In [41]:
from sklearn import metrics 
print("Decision Tree Classification gini model accuracy(in %):", metrics.accuracy_score(y_dec_test, y_dec_pred)*100)

Decision Tree Classification gini model accuracy(in %): 59.09090909090909


In [42]:
clf_entropy = DecisionTreeClassifier(criterion = "entropy", random_state = 100,
                 max_depth=3, min_samples_leaf=5)
clf_entropy.fit(X_dec_train, y_dec_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=5, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=100,
            splitter='best')

In [43]:
y_dec_pred_en = clf_entropy.predict(X_dec_test)

In [44]:
from sklearn import metrics 
print("Decision Tree Classification Entropy model accuracy(in %):", metrics.accuracy_score(y_dec_test, y_dec_pred_en)*100)

Decision Tree Classification Entropy model accuracy(in %): 59.09090909090909


In [45]:
from sklearn.ensemble import RandomForestClassifier

In [46]:
rfc = RandomForestClassifier(n_estimators=30)

In [47]:
rfc.fit(X_dec_train, y_dec_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=30, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [48]:
y_dec_pred = rfc.predict(X_dec_test)

In [49]:
from sklearn import metrics 
print("Random Forest Classification model accuracy(in %):", metrics.accuracy_score(y_dec_test, y_dec_pred_en)*100)

Random Forest Classification model accuracy(in %): 59.09090909090909


In [50]:
from xgboost.sklearn import XGBClassifier

In [51]:
xgbc=XGBClassifier(learning_rate =0.1, n_estimators=1000, max_depth=8, min_child_weight=6, gamma=0.1, subsample=0.9,colsample_bytree=0.95,reg_alpha=2, objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27)

In [52]:
from sklearn.preprocessing import scale
X_data_scale=scale(X_dec[['AMOUNT', 'DATE2']])
X_data_scale

array([[-0.30993983, -1.52024781],
       [-0.35275436, -1.52024781],
       [-0.38658623, -1.52024781],
       [-0.41571689, -1.52024781],
       [-0.41319839, -1.52024781],
       [-0.40304043, -1.52024781],
       [ 3.77003259, -1.30652597],
       [-0.40707843, -1.30652597],
       [ 1.33548115, -1.09280413],
       [-0.12441022, -1.09280413],
       [ 1.2515311 , -1.09280413],
       [ 0.4120306 , -1.09280413],
       [-0.0916697 , -0.66536045],
       [-0.41798354, -0.66536045],
       [-0.15882974, -0.66536045],
       [-0.40866509, -0.66536045],
       [-0.36719376, -0.66536045],
       [-0.41521319, -0.66536045],
       [-0.41999834, -0.66536045],
       [-0.38549487, -0.55849953],
       [-0.4272063 , -0.45163861],
       [-0.4272063 , -0.34477769],
       [ 1.16758105, -0.23791677],
       [-0.34351985, -0.23791677],
       [-0.33346095, -0.23791677],
       [ 3.68608254, -0.13105585],
       [-0.41864759, -0.13105585],
       [-0.38549487, -0.02419493],
       [-0.40593587,

In [53]:
from sklearn.preprocessing import StandardScaler
slc= StandardScaler()

In [54]:
X_data_std = slc.fit_transform(X_data_scale)
X_data_std

array([[-0.30993983, -1.52024781],
       [-0.35275436, -1.52024781],
       [-0.38658623, -1.52024781],
       [-0.41571689, -1.52024781],
       [-0.41319839, -1.52024781],
       [-0.40304043, -1.52024781],
       [ 3.77003259, -1.30652597],
       [-0.40707843, -1.30652597],
       [ 1.33548115, -1.09280413],
       [-0.12441022, -1.09280413],
       [ 1.2515311 , -1.09280413],
       [ 0.4120306 , -1.09280413],
       [-0.0916697 , -0.66536045],
       [-0.41798354, -0.66536045],
       [-0.15882974, -0.66536045],
       [-0.40866509, -0.66536045],
       [-0.36719376, -0.66536045],
       [-0.41521319, -0.66536045],
       [-0.41999834, -0.66536045],
       [-0.38549487, -0.55849953],
       [-0.4272063 , -0.45163861],
       [-0.4272063 , -0.34477769],
       [ 1.16758105, -0.23791677],
       [-0.34351985, -0.23791677],
       [-0.33346095, -0.23791677],
       [ 3.68608254, -0.13105585],
       [-0.41864759, -0.13105585],
       [-0.38549487, -0.02419493],
       [-0.40593587,

In [55]:
X_dec_train_std, X_dec_test_std, y_dec_train, y_dec_test = train_test_split(X_data_std, y_dec, test_size = .4, random_state=25)

In [57]:
xgbc.fit(X_dec_train_std,y_dec_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.95, gamma=0.1, learning_rate=0.1,
       max_delta_step=0, max_depth=8, min_child_weight=6, missing=None,
       n_estimators=1000, n_jobs=1, nthread=4, objective='binary:logistic',
       random_state=0, reg_alpha=2, reg_lambda=1, scale_pos_weight=1,
       seed=27, silent=True, subsample=0.9)

In [2]:
y_dec_pred=xgbc.predict(X_dec_test_std)

NameError: name 'xgbc' is not defined

In [59]:
from sklearn import metrics 
print("XGB Classification model accuracy(in %):", metrics.accuracy_score(y_dec_test, y_dec_pred)*100)

XGB Classification model accuracy(in %): 77.27272727272727
